In [3]:
from envs.minimal_jsp_env.util.jsp_generation.random_generator import RandomJSPGenerator, RandomJSPGeneratorPool, RandomJSPGeneratorOperationDistirbution, RandomJSPGeneratorWithJobPool
from envs.minimal_jsp_env.entities import Operation, JSPInstance
import json

from envs.minimal_jsp_env.util.jsp_conversion.readers import JSPReaderJSON
from envs.minimal_jsp_env.util.jsp_conversion.writers import JSPWriterJSON

### Reader / Writer

In [12]:
def get_job_matrices(jsp_instance: JSPInstance, display: bool = True):
    machine_types = []
    durations = []
    for job in jsp_instance.jobs:
        machine_types.append([operation.machine_type for operation in job])
        durations.append([operation.duration for operation in job])
        if display:
            print([f"({operation.machine_type}, {operation.duration})" for operation in job])
    
    return machine_types, durations

In [4]:
test_instance = RandomJSPGenerator(num_jobs=3, num_operations=3, max_op_duration=9).generate()
test_instance.jobs

[[Operation(job_id=0, op_id=0, machine_type=1, duration=9),
  Operation(job_id=0, op_id=1, machine_type=0, duration=7),
  Operation(job_id=0, op_id=2, machine_type=0, duration=3)],
 [Operation(job_id=1, op_id=0, machine_type=0, duration=2),
  Operation(job_id=1, op_id=1, machine_type=2, duration=6),
  Operation(job_id=1, op_id=2, machine_type=0, duration=1)],
 [Operation(job_id=2, op_id=0, machine_type=0, duration=1),
  Operation(job_id=2, op_id=1, machine_type=1, duration=7),
  Operation(job_id=2, op_id=2, machine_type=0, duration=9)]]

In [10]:
or_instance = []
for job in test_instance.jobs:
    job_ops = [[i.machine_type, i.duration] for i in job]
    or_instance.append(job_ops)
or_instance

[[[1, 9], [0, 7], [0, 3]], [[0, 2], [2, 6], [0, 1]], [[0, 1], [1, 7], [0, 9]]]

In [5]:
JSPWriterJSON().write_instance(test_instance, "test.json", "test")

In [6]:
JSPReaderJSON().read_instance('test.json').jobs

[[Operation(job_id=0, op_id=0, machine_type=1, duration=9),
  Operation(job_id=0, op_id=1, machine_type=0, duration=7),
  Operation(job_id=0, op_id=2, machine_type=0, duration=3)],
 [Operation(job_id=1, op_id=0, machine_type=0, duration=2),
  Operation(job_id=1, op_id=1, machine_type=2, duration=6),
  Operation(job_id=1, op_id=2, machine_type=0, duration=1)],
 [Operation(job_id=2, op_id=0, machine_type=0, duration=1),
  Operation(job_id=2, op_id=1, machine_type=1, duration=7),
  Operation(job_id=2, op_id=2, machine_type=0, duration=9)]]

### Dataset generation

In [45]:
import os

In [38]:
# entropies for datasets with pool_size = 36

entropy0_2 = [0.5, 0.5]
entropy0_3 = [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
entropy0_4 = [0.08333333333333333, 0.16666666666666666, 0.25, 0.08333333333333333, 0.4166666666666667]
entropy0_5 = [0.3333333333333333, 0.16666666666666666, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.16666666666666666]
entropy0_6 = [0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.16666666666666666, 0.08333333333333333, 0.16666666666666666, 0.16666666666666666, 0.08333333333333333]
entropy0_7 = [0.05555555555555555, 0.1111111111111111, 0.027777777777777776, 0.05555555555555555, 0.05555555555555555, 0.16666666666666666, 0.16666666666666666, 0.05555555555555555, 0.027777777777777776, 0.027777777777777776, 0.08333333333333333, 0.027777777777777776, 0.05555555555555555, 0.05555555555555555, 0.027777777777777776]
entropy0_8 = [0.16666666666666666, 0.027777777777777776, 0.027777777777777776, 0.027777777777777776, 0.027777777777777776, 0.027777777777777776, 0.05555555555555555, 0.027777777777777776, 0.05555555555555555, 0.027777777777777776, 0.027777777777777776, 0.05555555555555555, 0.05555555555555555, 0.027777777777777776, 0.027777777777777776, 0.05555555555555555, 0.027777777777777776, 0.08333333333333333, 0.05555555555555555, 0.08333333333333333, 0.027777777777777776]

In [46]:
number_of_instances = 100

In [47]:
for entropy_val in range(2, 9, 1):

    dir_path = f"data/entropy_datasets/entropy0_{entropy_val}/"
    os.makedirs(dir_path, exist_ok=True)

    for i in range(number_of_instances):
        instance = RandomJSPGeneratorOperationDistirbution(num_jobs=6, num_operations=6, max_op_duration=9).generate(eval(f"entropy0_{entropy_val}"))
        file_name = f"entropy0_{entropy_val}_{i}"
        JSPWriterJSON().write_instance(instance, f"{dir_path}/{file_name}", file_name)
        

[0.5, 0.5]
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
[0.08333333333333333, 0.16666666666666666, 0.25, 0.08333333333333333, 0.4166666666666667]
[0.3333333333333333, 0.16666666666666666, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.16666666666666666]
[0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.16666666666666666, 0.08333333333333333, 0.16666666666666666, 0.16666666666666666, 0.08333333333333333]
[0.05555555555555555, 0.1111111111111111, 0.027777777777777776, 0.05555555555555555, 0.05555555555555555, 0.16666666666666666, 0.16666666666666666, 0.05555555555555555, 0.027777777777777776, 0.027777777777777776, 0.08333333333333333, 0.027777777777777776, 0.05555555555555555, 0.05555555555555555, 0.027777777777777776]
[0.16666666666666666, 0.027777777777777776, 0.027777777777777776, 0.027777777777777776, 0.027777777777777776, 0.027777777777777776, 0.05555555555555555, 0.027777777777777776, 0.0555

### Duplicates check

In [7]:
JSPReaderJSON().read_instance('data/entropy_datasets/entropy0_2/entropy0_2_21').jobs

[[Operation(job_id=0, op_id=0, machine_type=4, duration=7),
  Operation(job_id=0, op_id=1, machine_type=3, duration=3),
  Operation(job_id=0, op_id=2, machine_type=4, duration=7),
  Operation(job_id=0, op_id=3, machine_type=4, duration=7),
  Operation(job_id=0, op_id=4, machine_type=3, duration=3),
  Operation(job_id=0, op_id=5, machine_type=3, duration=3)],
 [Operation(job_id=1, op_id=0, machine_type=4, duration=7),
  Operation(job_id=1, op_id=1, machine_type=4, duration=7),
  Operation(job_id=1, op_id=2, machine_type=3, duration=3),
  Operation(job_id=1, op_id=3, machine_type=4, duration=7),
  Operation(job_id=1, op_id=4, machine_type=4, duration=7),
  Operation(job_id=1, op_id=5, machine_type=3, duration=3)],
 [Operation(job_id=2, op_id=0, machine_type=3, duration=3),
  Operation(job_id=2, op_id=1, machine_type=4, duration=7),
  Operation(job_id=2, op_id=2, machine_type=4, duration=7),
  Operation(job_id=2, op_id=3, machine_type=4, duration=7),
  Operation(job_id=2, op_id=4, machine